In [ ]:
#| default_exp layers

In [ ]:
#| export
from typing import Any, Callable, Sequence, Union

import jax
from jax import lax, random, numpy as jnp
import flax.linen as nn
from fxlayers.initializers import *

In [ ]:
#| export
class GaborGammaFourier(nn.Module):
    features: int
    fs: int  = 1
    use_bias: bool = False
    norm_energy: bool = True
    train_A: bool = True

    @nn.compact
    def __call__(self,
                 inputs,
                 train=False,
                 **kwargs):
        freq = self.param("freq",
                          nn.initializers.uniform(scale=self.fs/2),
                          (self.features,))
        gammax = self.param("gammax",
                            nn.initializers.uniform(scale=1.),
                            (self.features,))
        gammay = self.param("gammay",
                            # nn.initializers.uniform(scale=1.),
                            equal_to(gammax*1.2),
                            (self.features,))
        theta = self.param("theta",
                           nn.initializers.uniform(scale=jnp.pi),
                           (self.features,))
        if self.train_A: A = self.param("A",
                                        nn.initializers.ones_init(),
                                        (inputs.shape[-1], self.features))
        else: A = jnp.array([1.])
        if self.use_bias: bias = self.param("bias",
                                            self.bias_init,
                                            (self.features,))
        else: bias = 0.

        fx, fy = self.generate_dominion(inputs.shape[1:-1], fs=self.fs)
        kernel = jax.vmap(self.gaussians, in_axes=(None,None,0,None,0,0,0), out_axes=-1)(fx, fy, freq, 0., gammax, gammay, theta)
        if self.norm_energy:
            kernel = kernel / (kernel**2).sum(axis=(1,2), keepdims=True)**(1/2)
            kernel = kernel / inputs.shape[-1]
            
        ## Add empty dims for broadcasting
        outputs = inputs[...,None] * kernel[None,...,None,:] * A[None,None,None,...]
        ## Sum the broadcasted dim
        outputs = outputs.sum(axis=-2)
        ## Add bias (or not)
        return outputs + bias
    
    @staticmethod
    def gaussians(fx, fy, fxm, fym, gammax, gammay, theta):
        def gaussian(fx, fy, fxm, fym, gammax, gammay):
            return jnp.exp(-((gammax**2)*(fx-fxm)**2 + (gammay**2)*(fy-fym)**2)/2)
        def rotate_dom(fx, fy, theta):
            fx_r = fx*jnp.cos(theta) - fy*jnp.sin(theta)
            fy_r = fx*jnp.sin(theta) + fy*jnp.cos(theta)
            return fx_r, fy_r
        # fxm_r, fym_r = rotate_dom(fxm, fym, theta)
        fxm_r, fym_r = fxm, fym
        fx, fy = rotate_dom(fx, fy, theta)
        # gammax, _ = rotate_dom(gammax, 0, -theta)
        # _, gammay = rotate_dom(0, gammay, -theta)
        
        return gaussian(fx, fy, fxm_r, fym_r, gammax, gammay) + gaussian(fx, fy, -fxm_r, -fym_r, gammax, gammay)

    @staticmethod
    def generate_dominion(input_size, fs):
        return jnp.meshgrid(jnp.linspace(-fs/2,fs/2,num=input_size[1]), jnp.linspace(-fs/2,fs/2,num=input_size[0]))
    
    def return_kernel(self, params, c_in, kernel_shape):
        fx, fy = self.generate_dominion(kernel_shape, fs=self.fs)
        kernel = jax.vmap(self.gaussians, in_axes=(None,None,0,None,0,0,0), out_axes=-1)(fx, fy, params["freq"], 0., params["gammax"], params["gammay"], params["theta"])
        return kernel

In [ ]:
#| export
class FourierPooling(nn.Module):
    reduction: float

    @nn.compact
    def __call__(self,
                 inputs,
                 **kwargs,
                 ):
        b, h, w, c = inputs.shape
        center = (h//2, w//2)
        h_f, w_f = int(h*self.reduction), int(w*self.reduction)
        return inputs[:,center[0]-h_f//2:center[0]+h_f//2,center[0]-w_f//2:center[0]+w_f//2,:]

In [ ]:
#| exporti
class GaborReductionBlock(nn.Module):
    features: int
    reduction: float
    fs: Union[int, float]
    norm_energy: bool = True
    same_real_imag: bool = True
    train_A: bool = True
    
    @nn.compact
    def __call__(self,
                 inputs_r,
                 inputs_i,
                 **kwargs,
                 ):
        ## Apply Gabors
        if not self.same_real_imag:
            outputs_r = GaborGammaFourier(features=self.features, fs=self.fs, norm_energy=self.norm_energy, train_A=self.train_A)(inputs_r)
            outputs_i = GaborGammaFourier(features=self.features, fs=self.fs, norm_energy=self.norm_energy, train_A=self.train_A)(inputs_i)
        else:
            g = GaborGammaFourier(features=self.features, fs=self.fs, norm_energy=self.norm_energy, train_A=self.train_A)
            outputs_r = g(inputs_r)
            outputs_i = g(inputs_i)
        
        ## Apply a dimensionality reduction step
        outputs_r = FourierPooling(reduction=self.reduction)(outputs_r)
        outputs_i = FourierPooling(reduction=self.reduction)(outputs_i)

        return outputs_r, outputs_i